# For a given day, download and plot SMAP L2 and L3 fields in the Beaufort Sea

Uses cloud datasets - currently written to run on a local machine (and download files to local) but could be easily adapted to run on the cloud

k.drushka // March 2022

In [2]:
# imports
import os
import requests
import numpy as np
# import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
import glob as glob

# from datetime import datetime

# plotting packages
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as mticker



# cloud stuff - ignore for now for working locally
# import s3fs

In [3]:
# ----- INPUTS ------
# define date and domain of interest

start_date = "2021-09-01T00:00:00"
end_date = "2021-09-01T23:59:59" # one day 
end_date = "2021-09-02T23:59:59" # multiple days 
date_range = start_date + "/" + end_date
print(date_range)

# beaufort sea. Use -180:180 longitude convention
lonrange = [-160, -130]
latrange = [68, 80]
bounding_box = (f'{lonrange[0]},{latrange[0]},{lonrange[1]},{latrange[1]}') # make this a string

# local data folder
datadir = '/data1/sassie/satellite/'

2021-09-01T00:00:00/2021-09-02T23:59:59


### L3 SMAP SSS. 

Dataset notes:
* SMAP data are now in the cloud - but we are working on our local machine, so download it anyway. 
* L3 JPL SMAP 8-day running mean is concept id C2208422957-POCLOUD (obtained from [earthdatasearch](https://search.earthdata.nasa.gov/search?q=SMAP%20Level%202B%20JPL&ff=Available%20from%20AWS%20Cloud))
* JPL SMAP is better near sea ice, so use that rather than RSS SMAP.
* note that JPL and RSS use different conventions for variable names (e.g., lat/lon/smap_sss vs latitude/longitude/sss), so code will have to be modified if RSS is to be use if both products are desired


This code requires that the user's .netrc file includes a PO.DAAC tools password, which is accessed here: https://podaac-tools.jpl.nasa.gov/drive/ :
```
machine podaac-tools.jpl.nasa.gov \
login USERNAME\
password YOUR_PW
```

In [4]:
# --- DATASET INFO ---
# SMAP JPL L3 V5.0
concept_id = 'C2208422957-POCLOUD' 

# base URL for NASA earthdata search: 
granule_url = 'https://cmr.earthdata.nasa.gov/search/granules'

# use "requests" to find the granules in our date range & domain 
response = requests.get(granule_url,
                       params={
                           'concept_id': concept_id,
                           'temporal': date_range,  
                           'bounding_box': bounding_box,
                           'page_size': 200,
                       },
                       headers={
                           'Accept': 'application/json'
                       }
                      )
print(response.headers['CMR-Hits'])  # number of granules identified

10


In [8]:
granules

[{'boxes': ['-90 -180 90 180'],
  'time_start': '2021-08-24T12:00:00.000Z',
  'updated': '2022-02-11T20:49:52.633Z',
  'dataset_id': 'JPL SMAP Level 3 CAP Sea Surface Salinity Standard Mapped Image 8-Day Running Mean V5.0 Validated Dataset',
  'data_center': 'POCLOUD',
  'title': 'SMAP_L3_SSS_20210828_8DAYS_V5.0',
  'coordinate_system': 'CARTESIAN',
  'day_night_flag': 'UNSPECIFIED',
  'time_end': '2021-09-01T12:00:00.000Z',
  'id': 'G2215658221-POCLOUD',
  'original_format': 'UMM_JSON',
  'granule_size': '10.870092391967773',
  'browse_flag': False,
  'collection_concept_id': 'C2208422957-POCLOUD',
  'online_access_flag': True,
  'links': [{'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#',
    'title': 'Download SMAP_L3_SSS_20210828_8DAYS_V5.0.nc',
    'hreflang': 'en-US',
    'href': 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2021/236/SMAP_L3_SSS_20210828_8DAYS_V5.0.nc'},
   {'rel': 'http://esipfed.org/ns/fedsearc

In [7]:
# ---- DOWNLOAD DATA ----
# create a local folder for the data if it doesn't exist
savedir = f'{datadir}smap/L3/'
os.makedirs(savedir, exist_ok=True)

# download the granules
granules = response.json()['feed']['entry']

# convert our start/end dates to datetime format:
start_date_dt = datetime.strptime(start_date,'%Y-%m-%dT%H:%M:%S').date()
end_date_dt = datetime.strptime(end_date,'%Y-%m-%dT%H:%M:%S').date() 
# list of granules we will download and load:
granules_load = []
for granule in granules:
    url = granule['links'][0]['href']
    id = granule['title'] + '.nc'
    r = requests.get(url)
    outfile = Path(savedir, id)
    # convert time_start from the file metadata to a datetime:
    thistime =  datetime.strptime(granule['time_start'],'%Y-%m-%dT%H:%M:%S.000Z').date() 
    # note! these are 8-day running means, so the search returns 9 files per day
    # only bother loading the days of interest (there's probably a nicer way to to this!)
    # - check if the granule if it is between start_date and end_date
    if (thistime >= start_date_dt) & (thistime <= end_date_dt):
        print(thistime, id)
        granules_load.append(f'{savedir}/{id}')
        # if the file doens't exist locally, download
        if (not outfile.exists()):
            print('downloading', url, id)
            with open(outfile, 'wb') as f:
                f.write(r.content)    
        else:
            print('already exists - skipping')
else:
    print(granule['title'] + ' out of time range, not downloading')

2021-09-01 SMAP_L3_SSS_20210905_8DAYS_V5.0.nc
already exists - skipping
2021-09-02 SMAP_L3_SSS_20210906_8DAYS_V5.0.nc
already exists - skipping
SMAP_L3_SSS_20210906_8DAYS_V5.0 out of time range, not downloading
